# Spin Glass - SamplerV2
This code can address up to 63-qubit instance on simulator.

In [1]:
import numpy as np
import pandas as pd
from hydra import initialize, compose
import dill
import os
import logging
import time
import matplotlib.pyplot as plt
plt.style.use('default')

import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_aer import AerProvider

from os.path import dirname, abspath
import sys
parent_dir = dirname(dirname(abspath(' ')))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from src.bitflip_strategy.Local_Search_PUBO import local_search, comp_objective
from src.optimizer.Quantum_Optimizer_FGSR_b_hardwareV2 import QuantumOptimizer
from src.optimizer.utils_logger import make_logger, initialize_data
from src.optimizer.hardware_utils import create_ansatz, get_initial_layout, dictconfig_to_dict
from Cplex_HuboSolver import CPLEX_HUBOsolver

import warnings
warnings.simplefilter('ignore')

In [2]:
# Load config file
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name='config_sampleV2.yaml')

In [3]:
# Set backend
env = cfg.backend.env
qiskit_version = int(qiskit.__version__.split(".")[0])

if qiskit_version < 1:
    from qiskit_ibm_runtime import Sampler
    from qiskit.primitives import BackendSampler
else:
    from qiskit_ibm_runtime import (
        SamplerV2 as Sampler,
        SamplerOptions
    )
    from qiskit.primitives import BackendSamplerV2 as BackendSampler

if cfg.backend.env == "hardware" or cfg.layout.use_real_backend:
    service = QiskitRuntimeService(**cfg.QiskitRuntimeService)

if cfg.layout.use_real_backend:
    layout_backend = service.get_backend(cfg.layout.name)
else:
    layout_backend = None

if cfg.backend.env == "hardware":
    backend = service.get_backend(cfg.backend.name)
    layout_backend = backend
else:
    backend = AerProvider().get_backend(cfg.backend.name)
    service = None

In [4]:
# Initialize data and logger
output_dir, idx_set, theta, x_ini, fval, indval, df_job_id, df_result = initialize_data(cfg)
logger = make_logger(cfg, output_dir, backend, env)

Parameters:
	instance:
		dir = ../instances/SpinGlass
		list = ['ibmq_guadalupe_0', 'ibm_geneva_0']
		trial_list = [0]
	optimizer:
		search = True
		n_flips = 50
		n_reps = 2
		alpha = 0.4
		random_update = True
		max_epoch = 1
		restart_s = 0.8
		restart_std = 0.0
		early_stop = False
		mixing_ratio = 0
		refval = None
		intermediate_result_display = True
		save_counts = False
	options:
		save_result = True
		output_dir = results
		timezone = Asia/Tokyo
		verbose = True
		verbose_optimization = False
		restart = False
		date = None
	backend:
		env = simulator
		name = aer_simulator_matrix_product_state
	layout:
		use_real_backend = True
		name = ibm_torino
		automatic_selection = True
		start_qubit = 14
	QiskitRuntimeService:
		channel = ibm_quantum
		instance = client-enablement/its/its-skill-buildi
	sampler_options:
		default_shots = 10000
		dynamical_decoupling = {'enable': True, 'sequence_type': 'XpXm'}
	transpile:
		optimization_level = 3


output_dir: results/20240817-202107


In [5]:
idx_set

[]

In [6]:
# Run optimization
logger.info("Backend Info:")
logger.info(f"\tbackend = {backend}")
logger.info(f"\tlayout_backend = {layout_backend}")
instance_list = cfg.instance.list
logger.info("Problem Info:")
logger.info(f"\tinstance_list = {instance_list}")
shots = cfg.sampler_options.default_shots
sampler_options = dictconfig_to_dict(cfg.sampler_options)

result_data = []
for instance in instance_list:
    # Load instance data
    with open(f'{cfg.instance.dir}/{instance}_hubo.pickle', mode='br') as fi:
        data = dill.load(fi)

    # Convert data to pubo format
    pubo_terms = {}
    offset = 0
    qubits_set = set()
    ord = -1
    for key, value in data.items():
        if key == -1:
            offset = value
            continue
        pubo_terms[key] = value
        ord = max(ord, len(key))
        for i in key:
            qubits_set.add(i)
    num_qubits = len(qubits_set)
    obj_data = [num_qubits, pubo_terms]
    cplex_result = CPLEX_HUBOsolver().solve(obj_data)
    refval = cplex_result.fval + offset

    pubo_data = []
    for key in data.keys():
        if key == -1:
            pubo_offset = data[key]
        else:
            l = list(key)
            i = ord - len(l)
            if i > 0:
                for _ in range(i):
                    l.append(l[-1])
            l.append(data[key])
            pubo_data.append(l)
    pubo_data = np.asarray(pubo_data, dtype=float)

    # Define objective
    objective = lambda x: comp_objective(x, ord, pubo_data, pubo_offset)
    def search(x):
        x = local_search(x, ord, pubo_data, pubo_offset, cfg.optimizer.n_flips)
        return np.asarray(x, dtype=int)

    logger.info("-"*130)
    logger.info(f"instance = {instance} \t num_qubits = {num_qubits} \t refval = {refval}")
    logger.info("-"*130)

    for trial in cfg.instance.trial_list:
        np.random.seed(trial)
        if output_dir is not None:
            output_dir_pb_trial = f"{output_dir}/{instance}_trial{trial}"
            if os.path.exists(output_dir_pb_trial) == False:
                os.mkdir(output_dir_pb_trial)
            with open(f"{output_dir_pb_trial}/pubo_data.pickle", 'wb') as f:
                dill.dump(pubo_data, f)
        logger_pb = logging.getLogger(f"SpinGlass Optimization Log: (instance, shots, trial) = {(instance, shots, trial)}")
        logger_pb.setLevel(logging.DEBUG)
        if cfg.options.verbose_optimization:
            st_handler_pb = logging.StreamHandler()
            st_handler_pb.setLevel(logging.DEBUG)
            logger_pb.addHandler(st_handler_pb)
        if output_dir is not None:
                fl_handler_pb = logging.FileHandler(filename=f"{output_dir_pb_trial}/optimization.log", mode="a")
                fl_handler_pb.setLevel(logging.DEBUG)
                logger_pb.addHandler(fl_handler_pb)
        logger_pb.info(f"instance: {instance} \t refval = {refval:.3f} \t shots = {shots} \t trial = {trial}")
        
        # create initial_layout
        chain_nodes = [i for i in range(num_qubits)]
        sep_nodes = []
        sep_nodes_coupling = []
        if layout_backend is not None:
            if cfg.layout.automatic_selection:
                chain_nodes, sep_nodes, sep_nodes_coupling = get_initial_layout(layout_backend, num_qubits, cfg.layout.start_qubit)
                initial_layout = chain_nodes + sep_nodes
            else:
                initial_layout = cfg.initial_layout[cfg.layout.name][:num_qubits]
        else:
            initial_layout = None
        layout_data = {
            "initial_layout": initial_layout,
            "chain_nodes": chain_nodes,
            "sep_nodes": sep_nodes,
            "sep_nodes_coupling": sep_nodes_coupling
        }
        if output_dir is not None:
            with open(f"{output_dir_pb_trial}/layout_data.pickle", 'wb') as f:
                dill.dump(layout_data, f)
        sub_qc = create_ansatz(cfg.optimizer.n_reps, chain_nodes, sep_nodes, sep_nodes_coupling)
        
        with Session(service=service, backend=backend) as session:
            sampler = Sampler(mode=session)
            sampler.options.update(**sampler_options)
            QuantumSolver = QuantumOptimizer()
            QuantumSolver.set_optimizer(num_qubits, sub_qc, sampler, layout_backend, cfg.transpile.optimization_level, initial_layout)
            QuantumSolver.run(
                objective = objective,
                search = search,
                alpha = cfg.optimizer.alpha,
                random_update = cfg.optimizer.random_update,
                max_epoch = cfg.optimizer.max_epoch,
                restart_s = cfg.optimizer.restart_s,
                restart_std = cfg.optimizer.restart_std,
                refval = refval,
                intermediate_result_display = cfg.optimizer.intermediate_result_display,
                early_stop = cfg.optimizer.early_stop,
                x_ini = x_ini,
                output_dir = output_dir_pb_trial,
                logger = logger_pb,
                seed = trial,
                idx_set = idx_set,
                df_job_id = df_job_id,
                df_result = df_result,
                theta = theta,
                mixing_ratio=cfg.optimizer.mixing_ratio,
                fval=fval,
                indval=indval,
                save_counts=cfg.optimizer.save_counts
            )
        step = len(QuantumSolver.result_history)
        objval = QuantumSolver.result_history[-1][3]
        expval = QuantumSolver.result_history[-1][4]
        sprob = QuantumSolver.result_history[-1][5]
        result_data.append([instance, num_qubits, refval, backend.name, shots, trial, step, objval, expval, sprob])
        time.sleep(5)
        logger.info(f"trial = {trial} \t step = {step} \t objval = {objval} \t cvar = {expval} \t s-prob = {sprob}")
        df = pd.DataFrame(data=result_data, columns=["instance", "num_qubits", "refval", "backend", "shots", "trial", "step", "objval", "cvar", "s-prob"])
        df.to_csv(f"{output_dir}/df_result_summary.csv", index=False)

Backend Info:
	backend = AerSimulator('aer_simulator_matrix_product_state')
	layout_backend = <IBMBackend('ibm_torino')>
Problem Info:
	instance_list = ['ibmq_guadalupe_0', 'ibm_geneva_0']
----------------------------------------------------------------------------------------------------------------------------------
instance = ibmq_guadalupe_0 	 num_qubits = 16 	 refval = -22.0
----------------------------------------------------------------------------------------------------------------------------------


	epoch = 0 	 step = 0 	 (j, k) = (0, 11) 	 objval = -22.000 	 cvar = -5.822   	 s-prob = 1.000e-04
	epoch = 0 	 step = 1 	 (j, k) = (0, 8) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 6.685e-01
	epoch = 0 	 step = 2 	 (j, k) = (1, 12) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 7.312e-01
	epoch = 0 	 step = 3 	 (j, k) = (2, 30) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.166e-01
	epoch = 0 	 step = 4 	 (j, k) = (3, 5) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.067e-01
	epoch = 0 	 step = 5 	 (j, k) = (4, 27) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.092e-01
	epoch = 0 	 step = 6 	 (j, k) = (5, 31) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.096e-01
	epoch = 0 	 step = 7 	 (j, k) = (6, 6) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.066e-01
	epoch = 0 	 step = 8 	 (j, k) = (7, 13) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.842e-01
	epoch = 0 	 step = 9 	 (j, k) = (8, 15) 	 objval = -22.000 	 cvar = -22.000   	 s-prob = 8.917e-01
	epo

trial = 0 	 step = 33 	 objval = 16 	 cvar = -22.0 	 s-prob = -22.0
----------------------------------------------------------------------------------------------------------------------------------
instance = ibm_geneva_0 	 num_qubits = 27 	 refval = -42.0
----------------------------------------------------------------------------------------------------------------------------------


	epoch = 0 	 step = 0 	 (j, k) = (0, 53) 	 objval = -42.000 	 cvar = -7.770   	 s-prob = 0.000e+00
	epoch = 0 	 step = 1 	 (j, k) = (0, 40) 	 objval = -42.000 	 cvar = -40.178   	 s-prob = 2.291e-01
	epoch = 0 	 step = 2 	 (j, k) = (1, 49) 	 objval = -42.000 	 cvar = -40.499   	 s-prob = 2.499e-01
	epoch = 0 	 step = 3 	 (j, k) = (2, 9) 	 objval = -42.000 	 cvar = -40.429   	 s-prob = 2.434e-01
	epoch = 0 	 step = 4 	 (j, k) = (3, 19) 	 objval = -42.000 	 cvar = -40.670   	 s-prob = 2.670e-01
	epoch = 0 	 step = 5 	 (j, k) = (4, 48) 	 objval = -42.000 	 cvar = -40.982   	 s-prob = 2.982e-01
	epoch = 0 	 step = 6 	 (j, k) = (5, 31) 	 objval = -42.000 	 cvar = -40.932   	 s-prob = 2.932e-01
	epoch = 0 	 step = 7 	 (j, k) = (6, 14) 	 objval = -42.000 	 cvar = -41.222   	 s-prob = 3.222e-01
	epoch = 0 	 step = 8 	 (j, k) = (7, 6) 	 objval = -42.000 	 cvar = -41.563   	 s-prob = 3.563e-01
	epoch = 0 	 step = 9 	 (j, k) = (8, 22) 	 objval = -42.000 	 cvar = -41.792   	 s-prob = 3.792e-01
	ep

trial = 0 	 step = 55 	 objval = 8 	 cvar = -42.0 	 s-prob = -42.0
